In [1]:
import requests
import urllib
from bs4 import BeautifulSoup
import re
import io
import gzip
import time
import csv
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import glob
import os
from os import path
import datetime
from dateutil.parser import parse
from selenium.webdriver.support.ui import Select


In [ ]:
df_equity = pd.read_csv("C:\\Users\\venu\\Desktop\\Stock Market Analysis\\Data\\Equity.csv")
security_numbers = df_equity["Security Code"].tolist()
security_n = df_equity["Security Name"].tolist()
security_names = dict(zip(security_numbers, security_n))

In [ ]:
def get_corporate_actions(nums):
    
    def get_date(d,m,y):
        yy = 2021
        while (int(y) < yy):
            yy -= 10
            year = driver.find_element_by_xpath('/html/body/div[2]/div/div/select[2]')
            segment = Select(year)
            segment.select_by_visible_text(str(yy))
        year = driver.find_element_by_xpath('/html/body/div[2]/div/div/select[2]')
        segment = Select(year)
        segment.select_by_visible_text(y)
        
        month = driver.find_element_by_xpath('/html/body/div[2]/div/div/select[1]')
        segment = Select(month)
        segment.select_by_visible_text(m)
        
        days = driver.find_element_by_xpath("//table/tbody/tr/td/a[text()="+str(d)+"]")
        days.click()
        
    
    def traverse_stock(security):
        if (path.exists("C:\\Users\\venu\\Desktop\\Stock Market Analysis\\Corporate Actions\\Corporate_Actions_" + security + ".csv")):
            print("present")
            df = pd.read_csv("C:\\Users\\venu\\Desktop\\Stock Market Analysis\\Corporate Actions\\Corporate_Actions_" + security + ".csv")
            fr_df = df.tail(1)['Ex Date']
            fr_date = fr_df.iloc[0]
            f = parse(fr_date)
            fd = f.date()
            if (fd == (datetime.datetime.today() - datetime.timedelta(days=1)).date()):
                return None
            else:
                print("not current date")
                months = {1 : "Jan", 2 : "Feb", 3 : "Mar", 4 : "Apr", 5 : "May", 6 : "Jun", 
                         7 : "Jul", 8 : "Aug", 9 : "Sep", 10 : "Oct", 11 : "Nov", 12 : "Dec"}
                day = 1 if (fd.day + 1 == 32) else fd.day + 1
                month = -1
                if (fd.day + 1 == 32):
                    month = fd.month + 1
                    if (month == 13):
                        month = 1
                else:
                    month = fd.month
                year = -1
                if (fd.day + 1 == 32 and fd.month + 1 == 13):
                    year = fd.year + 1
                else:
                    year = fd.year
                element = driver.find_element_by_xpath('//*[@id="ContentPlaceHolder1_SmartSearch_smartSearch"]')
                element.clear()
                element.send_keys(security)
                element.send_keys(Keys.ENTER)
                time.sleep(2)
                from_date = driver.find_element_by_xpath('//*[@id="ContentPlaceHolder1_txtDate"]')
                from_date.clear()
                from_date.click()
                get_date(str(day),months[month],str(year))
                time.sleep(2)
                to_date = driver.find_element_by_xpath('//*[@id="ContentPlaceHolder1_txtTodate"]')
                to_date.clear()
                to_date.click()
                cd = datetime.datetime.today()
                get_date(str(cd.day),months[cd.month],str(cd.year))
                time.sleep(2)
                submit = driver.find_element_by_xpath('//*[@id="ContentPlaceHolder1_btnSubmit"]')
                submit.click()
                time.sleep(2)
                store_stock(security)
                return None
        
        
        print("Intitial non presence")
        element = driver.find_element_by_xpath('//*[@id="ContentPlaceHolder1_SmartSearch_smartSearch"]')
        element.clear()
        element.send_keys(security)
        element.send_keys(Keys.ENTER)
        time.sleep(2)
        
        print("Not present")
        months = {1 : "Jan", 2 : "Feb", 3 : "Mar", 4 : "Apr", 5 : "May", 6 : "Jun", 
                 7 : "Jul", 8 : "Aug", 9 : "Sep", 10 : "Oct", 11 : "Nov", 12 : "Dec"}
        from_date = driver.find_element_by_xpath('//*[@id="ContentPlaceHolder1_txtDate"]')
        from_date.clear()
        from_date.click()
        get_date("01","Jan","2000")
        to_date = driver.find_element_by_xpath('//*[@id="ContentPlaceHolder1_txtTodate"]')
        to_date.clear()
        to_date.click()
#         cd = datetime.datetime.today()
#         get_date(str(cd.day),months[cd.month],str(cd.year))
        get_date("01", "Jan", "2010")
        submit = driver.find_element_by_xpath('//*[@id="ContentPlaceHolder1_btnSubmit"]')
        submit.click()
        time.sleep(5)
        store_stock(security)
        
    def remake_stock(security):
        files_list = os.listdir("C:\\Users\\venu\\Desktop\\Stock Market Analysis\\Corporate Actions")
        if ("Corporate_Actions_" + security +  ".csv") not in files_list:
            path = "C:\\Users\\venu\\Desktop\\Stock Market Analysis\\Corporate Actions\\" + "Corporate_Actions" + ".csv"
            stock = pd.read_csv(path)
            stock.to_csv("C:\\Users\\venu\\Desktop\\Stock Market Analysis\\Corporate Actions\\Corporate_Actions_" + security + ".csv", index=False)
            os.remove(path)
        else:
            path = "C:\\Users\\venu\\Desktop\\Stock Market Analysis\\Corporate Actions\\Corporate_Actions.csv"
            merge_files = []
            for file in files_list:
                if file.startswith("Corporate_Actions"):
                    merge_files.append(file)
            print(merge_files)
            pth = "C:\\Users\\venu\\Desktop\\Stock Market Analysis\\Corporate Actions\\"
            combined_csv = pd.concat([pd.read_csv(pth + merge_files[f]) for f in range(len(merge_files) - 1, -1, -1)])
            combined_csv.to_csv( pth + "Corporate_Actions_" + security + ".csv", index=False, encoding='utf-8-sig')
            os.remove(path)


    def store_stock(security):
        download_stock = driver.find_element_by_xpath("/html/body/div[1]/form/div[4]/div/div/div[2]/div/div/div[2]/a/i")
        download_stock.click()

        
    chromeOptions = webdriver.ChromeOptions()
    chromeOptions.add_experimental_option("prefs", {"download.default_directory": "C:\\Users\\venu\\Desktop\\Stock Market Analysis\\Corporate Actions"})
    driver = webdriver.Chrome(ChromeDriverManager().install(),chrome_options=chromeOptions)
    url = "https://www.bseindia.com/corporates/corporate_act.aspx"
    driver.get(url)
    traverse_stock(str(nums))
    time.sleep(8)
    driver.quit()
    remake_stock(str(nums))
